# Import

In [1]:
# import os
# os.environ['CUDA_LAUNCH_BLOCKING']='1'

In [2]:
import os
import numpy as np
import pandas as pd
import random
import shutil
import time
from savify import Savify
from savify.types import Type, Format, Quality
from savify.utils import PathHolder
from tqdm import tqdm


from IPython.display import clear_output

pd.set_option('display.max_columns', None)

In [3]:
import pytorch_lightning as pl
# your favorite machine learning tracking tool
from pytorch_lightning.loggers import WandbLogger

import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import random_split, DataLoader, TensorDataset, Dataset

from torchmetrics.classification import MulticlassF1Score, F1Score

from torchvision import transforms
import wandb

In [4]:
def set_seed(seed: int = 42) -> None:
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    # When running on the CuDNN backend, two further options must be set
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    # Set a fixed value for the hash seed
    os.environ["PYTHONHASHSEED"] = str(seed)
    print(f"Random seed set as {seed}")

set_seed(42)

Random seed set as 42


In [5]:
# For Savify
CLIENT_ID = '6e92d9cbe39342a6813b0248e3481bdf'
CLIENT_SECRET = '918ecb1b41064003a7fd2bd2a72766f6'

SONGS_PATH = 'data/songs'

# EDA

In [6]:
data = pd.read_csv('data/dataset.csv')
data = data.drop(columns=['Unnamed: 0'])
print(data.shape)
data.head(5)

(114000, 20)


,track_id,artists,album_name,track_name,popularity,duration_ms,explicit,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,track_genre
0,5SuOikwiRyPMVoIQDJUgSV,Gen Hoshino,Comedy,Comedy,73,230666,False,0.676,0.4610,1,-6.746,0,0.1430,0.0322,0.000001,0.3580,0.715,87.917,4,acoustic
1,4qPNDBW1i3p13qLCt0Ki3A,Ben Woodward,Ghost (Acoustic),Ghost - Acoustic,55,149610,False,0.420,0.1660,1,-17.235,1,0.0763,0.9240,0.000006,0.1010,0.267,77.489,4,acoustic
2,1iJBSr7s7jYXzM8EGcbK5b,Ingrid Michaelson;ZAYN,To Begin Again,To Begin Again,57,210826,False,0.438,0.3590,0,-9.734,1,0.0557,0.2100,0.000000,0.1170,0.120,76.332,4,acoustic
3,6lfxq3CG4xtTiEg7opyCyx,Kina Grannis,Crazy Rich Asians (Original Motion Picture Sou...,Can't Help Falling In Love,71,201933,False,0.266,0.0596,0,-18.515,1,0.0363,0.9050,0.000071,0.1320,0.143,181.740,3,acoustic
4,5vjLSffimiIP26QG5WcN2K,Chord Overstreet,Hold On,Hold On,82,198853,False,0.618,0.4430,2,-9.681,1,0.0526,0.4690,0.000000,0.0829,0.167,119.949,4,acoustic


Each genre category consists of 1000 songs (examples):

In [7]:
def check_col_info(df: pd.DataFrame, col: str) -> None:
    genre_count_mean = df[col].value_counts().mean()
    genre_count_std = df[col].value_counts().std()
    print(f'{col} count mean: {genre_count_mean}')
    print(f'{col} count std: {genre_count_std}')
    print(f'{col} count nunique: {df[col].nunique()}')

check_col_info(data, 'track_genre')

track_genre count mean: 1000.0
track_genre count std: 0.0
track_genre count nunique: 114


# Create Subset & Download Songs

Get 30 songs for each genre:

In [8]:
def generate_subset(df: pd.DataFrame, genres_quantity: int = 20, samples_in_genre: int = 160) -> pd.DataFrame:
    new_df = pd.DataFrame(columns=df.columns)
    
    end_iter = len(df)
    if genres_quantity is not None:
        end_iter = min(end_iter, genres_quantity*1000)
        
    for iter_1000 in range(0, end_iter, 1000):
        samples = df.iloc[iter_1000 : iter_1000 + samples_in_genre, :]
        new_df = pd.concat([new_df, samples])
    
    new_df.reset_index(drop=True, inplace=True)
    clear_output()
    return new_df

subset_df = generate_subset(data)
check_col_info(subset_df, 'track_genre')
subset_df.head(1)

track_genre count mean: 160.0
track_genre count std: 0.0
track_genre count nunique: 20


,track_id,artists,album_name,track_name,popularity,duration_ms,explicit,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,track_genre
0,5SuOikwiRyPMVoIQDJUgSV,Gen Hoshino,Comedy,Comedy,73,230666,False,0.676,0.461,1,-6.746,0,0.143,0.0322,0.000001,0.358,0.715,87.917,4,acoustic


In [9]:
import re
import string


song_blacklist = ['steampop', 'three_german_dances__k_605__no__3_in_c__trio__die_schlittenfahrt_', 'variations_on_an_original_theme__op__36__enigma___var__9__adagio__nimrod___arr__j__meisl_for_string_quartet_',
                  'divertimento_in_d_major__k__334__iii__menuetto___trio__arr__s__palmgren_for_piano_solo_', 'christmas_oratorio__bwv_248___pt__5___for_the_1st_sunday_in_the_new_year__no__51_terzetto___ach__wann_wird_die_zeit_erscheinen__',
                  'nutcracker_suite__op__71a__th__35__arr__for_piano_4_hands___iif__danses_caractéristiques__danse_des_mirlitons__moderato_assai', 'superman_and_passive_aggressive_man', 'let_me_down_slowly__slowed___reverb_']
artist_blacklist = ['gruzja', 'academy_of_st__martin_in_the_fields_sir_neville_marriner', 'edward_elgar_arioso_quartett_wien', 'wolfgang_amadeus_mozart_una_bourne',
                    'johann_sebastian_bach_gundula_janowitz_christa_ludwig_fritz_wunderlich_münchener_bach_orchester_karl_richter', 'pyotr_ilyich_tchaikovsky_martha_argerich_nicolas_economou', 'erikka_innes', 'fenekot']


def download_songs(df: pd.DataFrame, downloads_path: str, quality: Quality, download_format: Format) -> None:
    temp_folder = os.path.join(downloads_path, 'temp')
    
    if os.path.exists(temp_folder):
        shutil.rmtree(temp_folder)
    os.mkdir(temp_folder)
    
    new_df = df.copy()
    s = Savify(api_credentials=(CLIENT_ID, CLIENT_SECRET),
               path_holder=PathHolder(downloads_path=temp_folder),
               quality=quality,
               download_format=download_format)
    
    songs_to_remove = []
    for iter in tqdm(range(len(df))):
        track_id = new_df.at[iter, 'track_id']
        artist = re.sub(f'[{string.punctuation}{string.whitespace}]', '_', new_df.at[iter, 'artists'].lower())
        track_name = re.sub(f'[{string.punctuation}{string.whitespace}]', '_', new_df.at[iter, 'track_name'].lower())
        
        if artist in artist_blacklist and track_name in song_blacklist:
            songs_to_remove.append(iter)
            continue
        
        song_path = os.path.join(downloads_path, f'song_{artist}_{track_name}.mp3')

        if not os.path.exists(song_path):
            print(f'Artist name: {artist}')
            print(f'Song name: {track_name}')
            print(f'https://open.spotify.com/track/{track_id}')
            s.download(f'https://open.spotify.com/track/{track_id}')
            song = next(os.scandir(temp_folder))
            temp_song_path = os.path.join(temp_folder, song.name)
            shutil.move(temp_song_path, song_path)
            time.sleep(2)
            clear_output(wait=True)
        
        new_df.at[iter, 'song_path'] = song_path.replace('\\', '/')
    
    new_df = new_df.drop(songs_to_remove)
    new_df.reset_index(drop=True, inplace=True)
    print(f'Songs removed: {len(songs_to_remove)}')

    return new_df

In [10]:
subset_df = download_songs(df=subset_df,
                           downloads_path=SONGS_PATH,
                           quality= Quality.Q128K,
                           download_format=Format.MP3)
subset_df.head(1)

[INFO]	Checking for updates...
[INFO]	A new version of Savify is available, get the latest release here: https://github.com/LaurenceRawlings/savify/releases
100%|██████████| 3200/3200 [00:00<00:00, 5015.66it/s]

Songs removed: 9


,track_id,artists,album_name,track_name,popularity,duration_ms,explicit,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,track_genre,song_path
0,5SuOikwiRyPMVoIQDJUgSV,Gen Hoshino,Comedy,Comedy,73,230666,False,0.676,0.461,1,-6.746,0,0.143,0.0322,0.000001,0.358,0.715,87.917,4,acoustic,data/songs/song_gen_hoshino_comedy.mp3


In [11]:
import librosa
from transformers import Wav2Vec2Processor, Data2VecAudioModel


def add_embeddings(df: pd.DataFrame, samples_per_song:int=1, sample_rate: int=16000, duration: int=15):
    processor = Wav2Vec2Processor.from_pretrained("facebook/data2vec-audio-base-960h")
    embedding_generator = Data2VecAudioModel.from_pretrained("m-a-p/music2vec-v1")
    
    new_df = df.copy()
    
    if torch.cuda.is_available():
        embedding_generator.cuda()
    
    for idx in tqdm(range(len(new_df))):
        song_path = new_df.at[idx, 'song_path']
        embedding_path = song_path.replace('.mp3', '.npy')
        
        
        if not os.path.exists(embedding_path):
            song, _ = librosa.load(song_path, sr=sample_rate, duration=600)
            song_duration = int(song.shape[0] / sample_rate)
            begin_cut = random.randint(0, song_duration-duration-1)
            end_cut = begin_cut + duration
            cutted_song = song[begin_cut*sample_rate : end_cut*sample_rate]
            
            proccesed_song = processor(cutted_song, sampling_rate=sample_rate, return_tensors="pt")
            if torch.cuda.is_available():
                proccesed_song = proccesed_song.to(device='cuda')
            with torch.no_grad():
                outputs = embedding_generator(**proccesed_song, output_hidden_states=True)
            
            # embedding shape [layers, timesteps, feature dimenstion]
            embedding = torch.stack(outputs.hidden_states).squeeze()
            embedding_numpy = embedding.cpu().numpy()
            np.save(embedding_path, embedding_numpy)

        new_df.at[idx, 'embedding_path'] = embedding_path
    
    return new_df
        
subset_df = add_embeddings(subset_df)
subset_df.head(1)

100%|██████████| 3191/3191 [00:00<00:00, 10514.07it/s]


,track_id,artists,album_name,track_name,popularity,duration_ms,explicit,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,track_genre,song_path,embedding_path
0,5SuOikwiRyPMVoIQDJUgSV,Gen Hoshino,Comedy,Comedy,73,230666,False,0.676,0.461,1,-6.746,0,0.143,0.0322,0.000001,0.358,0.715,87.917,4,acoustic,data/songs/song_gen_hoshino_comedy.mp3,data/songs/song_gen_hoshino_comedy.npy


In [12]:
subset_df['track_genre'] = subset_df['track_genre'].astype('category')
print(dict(zip(subset_df['track_genre'].cat.codes, subset_df['track_genre'])))
subset_df['track_genre'] = subset_df['track_genre'].cat.codes

{0: 'acoustic', 1: 'afrobeat', 2: 'alt-rock', 3: 'alternative', 4: 'ambient', 5: 'anime', 6: 'black-metal', 7: 'bluegrass', 8: 'blues', 9: 'brazil', 10: 'breakbeat', 11: 'british', 12: 'cantopop', 13: 'chicago-house', 14: 'children', 15: 'chill', 16: 'classical', 17: 'club', 18: 'comedy', 19: 'country'}


# Learn Model

In [13]:
class MusicDataset(Dataset):

    def __init__(self, df, data_dir='.', num_classes=10):
        self.df = df
        self.data_dir = data_dir
        self.num_classes = num_classes

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        embedding_path = self.df.iloc[idx, :]['embedding_path']
        embedding_path = os.path.join(self.data_dir, embedding_path)

        embedding_numpy = np.load(embedding_path)
        embedding = torch.from_numpy(embedding_numpy)
        begin_cut = random.randint(0, embedding.size(-2) - 250)
        end_cut = begin_cut + 249
        embedding = embedding[:, begin_cut:end_cut, :]
        
        time_reduced_embedding = embedding.mean(-2)
        
        label = self.df.iloc[idx, :]['track_genre'].copy()
        label_one_hot = np.zeros((self.num_classes))
        label_one_hot[label] = 1
        label_one_hot = torch.tensor(label_one_hot)
        
        sample = [time_reduced_embedding, label_one_hot]

        return sample

In [14]:
class MusicDataModule(pl.LightningDataModule):
    def __init__(self, df, batch_size, num_classes, data_dir='.'):
        super().__init__()
        self.df = df
        self.batch_size = batch_size
        self.data_dir = data_dir
        self.num_classes = num_classes
    
    def prepare_data(self):
        pass
    
    def setup(self, stage=None):
        music_full = MusicDataset(df=self.df, data_dir=self.data_dir, num_classes=self.num_classes)
        train_size = int(0.7*len(music_full))
        test_size = len(music_full) - train_size
        self.music_train, self.music_test = random_split(music_full, [train_size, test_size])
    
    def train_dataloader(self):
        return DataLoader(self.music_train, batch_size=self.batch_size, shuffle=True)

    def val_dataloader(self):
        return DataLoader(self.music_test, batch_size=self.batch_size)

    def test_dataloader(self):
        return DataLoader(self.music_test, batch_size=self.batch_size)

In [15]:
class GenreNet(pl.LightningModule):
    def __init__(self, num_classes=10, learning_rate=1e-3):
        super().__init__()
        
        # log hyperparameters
        self.save_hyperparameters()
        
        self.conv = nn.Sequential(
            nn.Conv1d(in_channels=13, out_channels=1, kernel_size=1),
            nn.ReLU(),
        )
        
        self.head = nn.Sequential(
            nn.Linear(768, 224),
            nn.BatchNorm1d(224),
            nn.ReLU(),
            nn.Linear(224, 224),
            nn.BatchNorm1d(224),
            nn.ReLU(),
            nn.Dropout(p=0.9),
            nn.Linear(224, num_classes)
        )
        
        self.num_classes = num_classes
        self.learning_rate = learning_rate
        self.loss = nn.BCEWithLogitsLoss()
        self.f1 = MulticlassF1Score(num_classes=num_classes)
    
    def forward(self, x):
        x = self.conv(x)
        x = torch.squeeze(x)
        x = self.head(x)
        x = torch.squeeze(x)
        return x
    
    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = self.loss(y_hat, y)
        f1_score = self.f1(torch.argmax(y_hat, dim=1), torch.argmax(y, dim=1))
        
        self.log('train_loss', loss, on_step=True, on_epoch=True, logger=True)
        self.log('train_F1', f1_score, on_step=True, on_epoch=True, logger=True)
        return loss
    
    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = self.loss(y_hat, y)
        f1_score = self.f1(torch.argmax(y_hat, dim=1), torch.argmax(y, dim=1))

        self.log('val_loss', loss, prog_bar=True)
        self.log('val_F1', f1_score, prog_bar=True)
        return loss
    
    def test_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = self.loss(y_hat, y)
        f1_score = self.f1(torch.argmax(y_hat, dim=1), torch.argmax(y, dim=1))

        self.log('test_loss', loss, prog_bar=True)
        self.log('test_F1', f1_score, prog_bar=True)
        if batch_idx == 0:
            print('MODEL')
            print('----------------------------------------------------')
            print(self.conv)
            print(self.head)
            print('----------------------------------------------------')
        return loss
    
    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=self.learning_rate)
        return optimizer

In [16]:
dm = MusicDataModule(df=subset_df,
                    batch_size=60,
                    num_classes=20)
# To access the x_dataloader we need to call prepare_data and setup.
dm.prepare_data()
dm.setup()

# Samples required by the custom ImagePredictionLogger callback to log image predictions.
# val_samples = next(iter(dm.val_dataloader()))
# val_imgs, val_labels = val_samples[0], val_samples[1]
# val_imgs.shape, val_labels.shape

In [17]:
model = GenreNet(num_classes=dm.num_classes)

# Initialize wandb logger
wandb_logger = WandbLogger(project='genre-classification', job_type='train')

# Initialize Callbacks
early_stop_callback = pl.callbacks.EarlyStopping(monitor="val_loss")
checkpoint_callback = pl.callbacks.ModelCheckpoint()

# Initialize a trainer
trainer = pl.Trainer(max_epochs=30,
                     accelerator='gpu',
                     devices=1,
                     logger=wandb_logger,
                     callbacks=[checkpoint_callback],
                     log_every_n_steps=1,
                     )

# Train the model ⚡🚅⚡
trainer.fit(model, dm)

# Evaluate the model on the held-out test set ⚡⚡
trainer.test(dataloaders=dm.test_dataloader())

# Close wandb run
wandb.finish()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: leon-1402 (corgi-team). Use `wandb login --relogin` to force relogin


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name | Type              | Params
-------------------------------------------
0 | conv | Sequential        | 14    
1 | head | Sequential        | 228 K 
2 | loss | BCEWithLogitsLoss | 0     
3 | f1   | MulticlassF1Score | 0     
-------------------------------------------
228 K     Trainable params
0         Non-trainable params
228 K     Total params
0.912     Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

c:\Users\PnthrLeo\.conda\envs\image-processing\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:236: PossibleUserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 12 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
c:\Users\PnthrLeo\.conda\envs\image-processing\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:236: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 12 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Training: 0it [00:00, ?it/s]

...

`Trainer.fit` stopped: `max_epochs=30` reached.
c:\Users\PnthrLeo\.conda\envs\image-processing\lib\site-packages\pytorch_lightning\trainer\trainer.py:1386: UserWarning: `.test(ckpt_path=None)` was called without a model. The best model of the previous `fit` call will be used. You can pass `.test(ckpt_path='best')` to use the best model or `.test(ckpt_path='last')` to use the last model. If you pass a value, this warning will be silenced.
  rank_zero_warn(
Restoring states from the checkpoint path at .\genre-classification\328lvwst\checkpoints\epoch=29-step=1140.ckpt
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Loaded model weights from checkpoint at .\genre-classification\328lvwst\checkpoints\epoch=29-step=1140.ckpt
c:\Users\PnthrLeo\.conda\envs\image-processing\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:236: PossibleUserWarning: The dataloader, test_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_

Testing: 0it [00:00, ?it/s]

MODEL
----------------------------------------------------
Sequential(
  (0): Conv1d(13, 1, kernel_size=(1,), stride=(1,))
  (1): ReLU()
)
Sequential(
  (0): Linear(in_features=768, out_features=224, bias=True)
  (1): BatchNorm1d(224, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU()
  (3): Linear(in_features=224, out_features=224, bias=True)
  (4): BatchNorm1d(224, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (5): ReLU()
  (6): Dropout(p=0.9, inplace=False)
  (7): Linear(in_features=224, out_features=20, bias=True)
)
----------------------------------------------------


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│          test_F1          │    0.3441731035709381     │
│         test_loss         │    0.13891064504394415    │
└───────────────────────────┴───────────────────────────┘

epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇████
test_F1,▁
test_loss,▁
train_F1_epoch,▁▁▂▂▂▂▂▂▂▂▂▂▃▂▂▃▃▃▃▃▄▄▄▄▅▆▇▇██
train_F1_step,▂▁▂▁▂▂▂▂▃▂▁▄▂▃▂▂▂▂▃▂▃▁▃▃▃▃▃▄▃▃▄▄▅▆▆▅▅█▆█
train_loss_epoch,█▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁
train_loss_step,█▆▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁
trainer/global_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
val_F1,▁▁▁▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▃▃▃▃▄▅▅▆▆▇██
val_loss,█▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁
epoch,29
